## This code was ran in Kaggle and the model was exported from there. The data set link used to train the model is here: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri

In [ ]:
import os
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf
import pickle
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Folder Paths

In [ ]:
X_train = []
Y_train = []
image_size = 224
labels = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
trts = ['Training','Testing']

for i in labels:
    for t in trts:
        folderPath = os.path.join(f'../input/brain-tumor-classification-mri/{t}',i)
        for j in os.listdir(folderPath):
            img = cv2.imread(os.path.join(folderPath,j))
            img = cv2.resize(img,(image_size,image_size))
            X_train.append(img)
            Y_train.append(i)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
X_train,Y_train = shuffle(X_train,Y_train,random_state=101)
X_train.shape

# Train testing split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_train,Y_train,test_size=0.1,random_state=101)

#turning string labels to integer values then converting to binary matrix

y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train=y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test=y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

# Convolutional Neural Network

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape=(image_size,image_size,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512,activation = 'relu'))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(len(labels),activation='softmax'))

callbacks = [
    # Save the best model based on validation accuracy
    ModelCheckpoint('braintumor.keras', monitor='val_loss', save_best_only=True, verbose=1),
    
    # Stop training if validation accuracy is not improving for 5 epochs
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True),
    
    # Reduce learning rate if validation loss is not improving
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6, verbose=1)
]

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=20,validation_split=0.2, callbacks=callbacks,verbose=0)
model.save('braintumor.h5')
with open('/kaggle/working/braintumor.pkl', 'wb') as f:
    pickle.dump(model, f)

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4))
ax1.plot(history.history['loss'], label = 'loss')
ax1.plot(history.history['val_loss'], label = 'val_loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Categorical Crossentropy Loss')

ax2.plot(history.history['accuracy'], label = 'accuracy')
ax2.plot(history.history['val_accuracy'], label = 'val_accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')

plt.legend()
plt.grid(False)
plt.show()

# Prediction

In [ ]:
def result(filepathinput,m):
    img = cv2.imread(filepathinput)
    img = cv2.resize(img,(image_size,image_size))
    img_array = np.array(img)
    img_array = img_array.reshape(1,image_size,image_size,3)
#     img = image.load_img(filepathinput)
    plt.imshow(img,interpolation = 'nearest')
    plt.show()

    prediction = m.predict(img_array)

    indices = prediction.argmax()
    types = labels[indices]
    match types:
        case 'glioma_tumor':
            tumor = 'Glioma Tumor'
        case 'meningioma_tumor':
            tumor = 'Meningioma Tumor'
        case 'no_tumor':
            tumor = 'No Tumor'
        case 'pituitary_tumor':
            tumor = 'Pituitary Tumor'
        case _:
            tumor = 'Unable to identify'
            

    confidence = np.max(prediction) * 100
    print(f'Tumor Type: {tumor}        Confidence Level: {confidence:.2f}%')
    return prediction

## Use this cell to test accuracy of model

In [ ]:
x = rand.randint(1,100)
y = rand.choice(labels)
filepathinput = f'/kaggle/input/brain-tumor-classification-mri/Testing/{y}/image({x}).jpg'
res = result(filepathinput, model)
print(f'Actual Type: {y}')

for i in range(4):
    print(f'Probability of {labels[i]}: {res[0][i]:.6f}')